# Apache Kafka Streaming Analytics
In this notebook Apache Kafka is going to be used and analyzed with reference to the streaming performance using the twitter dataset.

In this case, we are going to use only one **Kafka Broker** that streams the data to the **Kafka Consumer**.

In [2]:
# Install the Python Client for Apache Kafka
!pip install confluent-kafka

     |████████████████████████████████| 2.7 MB 6.5 MB/s eta 0:00:01     |██████████████████████████▋     | 2.3 MB 6.5 MB/s eta 0:00:01


In [9]:
# Load dependencies and set constants
import matplotlib.pyplot as plt
import json

from confluent_kafka import Producer, Consumer, KafkaError

DATA_GENERATION_IN_MB = 1000 # ~ 1GB
DATASET_SIZE_MB = 10
TWITTER_DATA_PATH = "../data/dataset.json"
KAFKA_TOPIC_TWITTER = "twitter-stream"
CONSUMER_GROUP_ID = "twitter-consumers"

### Reminder: Running Kafka Architecture required
The following cells assume a running Apache Kafka Environment.

In [ ]:
# Produce the data / write it to the Kafka Cluster
producer_config = {
    "bootstrap.servers": "localhost:9092"
}
p = Producer(producer_config)

# Fill the topic with the specified amount of data
generation_steps = int(DATA_GENERATION_IN_MB / DATASET_SIZE_MB)
with open(TWITTER_DATA_PATH, "r") as dataset:
    for step in range(generation_steps):
        print(f"Executing data generation step {step}...")
        dataset.seek(0) # Jump back to first line  
 
        for tweet in dataset:
            try:
                p.produce(KAFKA_TOPIC_TWITTER, value=tweet)
            except BufferError:
                print('[INFO] Local producer queue is full (%d messages awaiting delivery): Trying again after flushing...\n' % len(p)) 
                p.poll(0)

                p.flush() # Send the messages in the queue

                # The tweet that caused the BufferError could not be sent, so after flushing, we can now send it
                p.produce(KAFKA_TOPIC_TWITTER, value=tweet)

In [ ]:
def get_kafka_stats(json_stats_bytes):
    """ Callback for the Apache Kafka Consumer Configuration which 
    requests performance metrics / statistics from Kafka. 
    
    :param json_stats: The JSON statistics data in bytes
    """
    # Decode the bytes into a python dictionary representing the JSON
    stats = json.loads(json_stats_bytes)
    # TODO: Check if we can actually use those statistics. If so, uncomment them in the consumer config
    
# Consume the data
consumer_config = {
    "bootstrap.servers": "localhost:9092",
    "group.id": CONSUMER_GROUP_ID,
    "client.id": "client-1",
    "stats_cb": get_kafka_stats,
#     "statistics.interval.ms": 20,
#     'api.version.request': True,
    "enable.auto.commit": True,
    "session.timeout.ms": 6000,
    "default.topic.config": {"auto.offset.reset": "smallest"}
}
c = Consumer(consumer_config)

c.subscribe([KAFKA_TOPIC_TWITTER])

try:
    while True:
        msg = c.poll(0.1)
        
        if msg is None:
            continue
        elif not msg.error():
            msg_count += 1
            print(f"Received message {msg_count}")
        
            # Display the received tweet
            #tweet_json = json.loads(msg.value())
            #print(json.dumps(tweet_json, indent=4, ensure_ascii=False, sort_keys=True))
        elif msg.error().code() == KafkaError._PARTITION_EOF:
            print("End of partition reached {}/{}".format(msg.topic(), msg.partition()))
        else:
            print("Error occured: {}".format(msg.error().str()))
except KeyBoardInterrupt:
    pass
finally:
    c.close()